In [2]:
from theano.sandbox import cuda
#cuda.use('gpu2')

In [4]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [5]:
batch_size = 64

In [36]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [37]:
# convert 3D tensor to 4D tensor
X_test = np.expand_dims(X_test, 1)
X_train = np.expand_dims(X_train, 1)
X_train.shape
X_test.shape

(10000, 1, 28, 28)

In [38]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [39]:
# convert train and test outputs to onehot vectors
y_train = onehot(y_train)
y_test = onehot(y_test)

In [43]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [44]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [45]:
# simple helper function to normalize the input
def norm_input(x): return (x-mean_px)/std_px

## Linear Model

In [46]:
# Simple Linear Model which can classify into 10 classes
def get_lin_model():
    # sequential apis to simply create stacked layer model
    # multilayer perceptron model
    model = Sequential([
            # Lambda function - takes input normalization 
            # function and input shape
            Lambda(norm_input, input_shape=(1, 28, 28)),
            Flatten(),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [47]:
lm = get_lin_model()
# Keras API to get the images in the batches
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
# takes data array and label arrays
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [48]:
lm.fit_generator(batches, batches.N, nb_epoch=1, 
                validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 10s - loss: 0.4299 - acc: 0.8728 - val_loss: 0.2978 - val_acc: 0.9148


In [52]:
# training accuracy < validation accuracy - we are underfitting

In [54]:
print(lm.optimizer.lr)
lm.optimizer.lr=0.1
lm.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

0.1
Epoch 1/1
60000/60000 [==============================] - 10s - loss: 0.3003 - acc: 0.9140 - val_loss: 0.2789 - val_acc: 0.9212


In [55]:
lm.optimizer.lr=0.01
lm.fit_generator(batches, batches.N, nb_epoch=4, 
                 validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 10s - loss: 0.2837 - acc: 0.9192 - val_loss: 0.2777 - val_acc: 0.9229
Epoch 2/4
60000/60000 [==============================] - 10s - loss: 0.2787 - acc: 0.9213 - val_loss: 0.2779 - val_acc: 0.9239
Epoch 3/4
60000/60000 [==============================] - 10s - loss: 0.2729 - acc: 0.9240 - val_loss: 0.2781 - val_acc: 0.9232
Epoch 4/4
60000/60000 [==============================] - 10s - loss: 0.2701 - acc: 0.9248 - val_loss: 0.2838 - val_acc: 0.9246


In [56]:
lm.summary()
# Weight Matrix dimensions:
# 784 x 10 : weights
# 28x28 --> 1x784 * 784x10 --> 1x10 + 1x10(biases) --> 1x10 output

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 1, 28, 28)     0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 784)           0           lambda_1[0][0]                   
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            7850        flatten_1[0][0]                  
Total params: 7850
____________________________________________________________________________________________________


## Single Dense Layer

In [59]:
def get_fc_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1,28,28)),
            # 1x28x28 --> 1x784
            Flatten(),
            # 1x1784 * (784x512) --> 1x512
            Dense(512, activation='softmax'),
            # 1x512 * (512x10) --> 1x10
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [60]:
fc = get_fc_model()

In [62]:
fc.fit_generator(batches, batches.N, nb_epoch=1, 
                validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 15s - loss: 1.5713 - acc: 0.8156 - val_loss: 1.0670 - val_acc: 0.8459


In [64]:
print(fc.optimizer.lr)
# currently the model is underfitting

<TensorType(float32, scalar)>


In [65]:
fc.optimizer.lr = 0.1

In [66]:
fc.fit_generator(batches, batches.N, nb_epoch=4, 
                validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 15s - loss: 0.8214 - acc: 0.8476 - val_loss: 0.6422 - val_acc: 0.8487
Epoch 2/4
60000/60000 [==============================] - 15s - loss: 0.5548 - acc: 0.8546 - val_loss: 0.4967 - val_acc: 0.8628
Epoch 3/4
60000/60000 [==============================] - 15s - loss: 0.4574 - acc: 0.8599 - val_loss: 0.4328 - val_acc: 0.8568
Epoch 4/4
60000/60000 [==============================] - 15s - loss: 0.4123 - acc: 0.8643 - val_loss: 0.4221 - val_acc: 0.8659


In [67]:
# still underfitting
fc.optimizer.lr = 0.01
fc.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 15s - loss: 0.3845 - acc: 0.8741 - val_loss: 0.3900 - val_acc: 0.8581
Epoch 2/4
60000/60000 [==============================] - 15s - loss: 0.3679 - acc: 0.8750 - val_loss: 0.3834 - val_acc: 0.8649
Epoch 3/4
60000/60000 [==============================] - 15s - loss: 0.3516 - acc: 0.8855 - val_loss: 0.3845 - val_acc: 0.8851
Epoch 4/4
60000/60000 [==============================] - 15s - loss: 0.3341 - acc: 0.8983 - val_loss: 0.3552 - val_acc: 0.9031


In [68]:
fc.summary()
# number of parameters have increase
# parameters = 784 x 512 + 512 x 10 + 10 + 512 = 407050

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 1, 28, 28)     0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 784)           0           lambda_2[0][0]                   
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           401920      flatten_2[0][0]                  
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 10)            5130        dense_2[0][0]                    
Total params: 407050
______________________________________________________________________

## Basic 'VGG-style' CNN

In [84]:
def get_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1, 28, 28)),
            Convolution2D(32, 3, 3, activation='relu'),
            Convolution2D(32, 3, 3, activation='relu'),
            MaxPooling2D(),
            Convolution2D(64, 3, 3, activation='relu'),
            Convolution2D(64, 3, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
        ])
    # compile the model with optimization function, type of loss function
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [85]:
model = get_model()

In [86]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 1, 28, 28)     0           lambda_input_5[0][0]             
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 32, 26, 26)    320         lambda_5[0][0]                   
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D) (None, 32, 24, 24)    9248        convolution2d_9[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 32, 12, 12)    0           convolution2d_10[0][0]           
___________________________________________________________________________________________

In [87]:
model.fit_generator(batches, batches.N, nb_epoch=1,
                   validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 162s - loss: 0.1091 - acc: 0.9659 - val_loss: 0.0505 - val_acc: 0.9824


In [89]:
model.optimizer.lr=0.1
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)model.optimizer.lr=0.01

Epoch 1/1
13312/60000 [=====>........................] - ETA: 123s - loss: 0.0374 - acc: 0.9884

KeyboardInterrupt: 

In [90]:
model.optimizer.lr=0.01
model.fit_generator(batches, batches.N, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/8
 2816/60000 [>.............................] - ETA: 151s - loss: 0.0494 - acc: 0.9847

KeyboardInterrupt: 

## Data Augmentation

In [91]:
model = get_model()
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [ ]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.1
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.01
model.fit_generator(batches, batches.N, nb_epoch=8, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.001
model.fit_generator(batches, batches.N, nb_epoch=14, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.0001
model.fit_generator(batches, batches.N, nb_epoch=10, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

## Batchnorm + data augmentation

In [92]:
def get_model_bn():
    model = Sequential([
            Lambda(norm_input, input_shape=(1, 28, 28)),
            Convolution2D(32, 3, 3, activation='relu'),
            # across all the axis
            BatchNormalization(axis=1),
            Convolution2D(32, 3, 3, activation='relu'),
            MaxPooling2D(),
            BatchNormalization(axis=1),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(64, 3, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            BatchNormalization(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [93]:
model = get_model_bn()

In [94]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
48128/60000 [=======================>......] - ETA: 41s - loss: 0.1799 - acc: 0.9450

KeyboardInterrupt: 

In [ ]:
model.optimizer.lr=0.1
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

In [ ]:
model.optimizer.lr=0.01
model.fit_generator(batches, batches.N, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.001
model.fit_generator(batches, batches.N, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

## Batchnorm + dropout + dataaugmentaion

In [95]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = get_model_bn_do()
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.1
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.01
model.fit_generator(batches, batches.N, nb_epoch=12, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)
model.optimizer.lr=0.001
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

## Ensembling

In [ ]:
def fit_model():
    model = get_model_bn_do()
    model.fit_generator(batches, batches.N, nb_epoch=1, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.optimizer.lr=0.1
    model.fit_generator(batches, batches.N, nb_epoch=4, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.optimizer.lr=0.01
    model.fit_generator(batches, batches.N, nb_epoch=12, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    model.optimizer.lr=0.001
    model.fit_generator(batches, batches.N, nb_epoch=18, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.N)
    return model

In [96]:
# get the array for model
models = [fit_model() for i range(6)]

SyntaxError: invalid syntax (<ipython-input-96-47b9fade01f0>, line 2)

In [97]:
path = '../../data/mnist/'
model_path = path + 'models/'

In [ ]:
for i,m in enumerate(models):
    m.save_weights(model_path+'cnn-mnist23-'+str(i)+'.pkl')

In [ ]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models])

In [ ]:
# mean across all the rows
evals.mean(axis=0)
# concatenate all the predictions
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models])B

In [ ]:
all_preds.shape

In [ ]:
avg_preds = all_preds.mean(axis=0)

In [ ]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()